In [1]:
import pandas as pd
import rdflib
from rdflib import Graph, Namespace, URIRef, BNode, Literal, RDF
from rdflib.namespace import NamespaceManager
import datetime
pd.options.display.max_colwidth = 144

In [2]:
nvt_master = pd.read_excel('./NVT_Metadatentabelle_MASTER_0207.xlsm', sheet_name=None)

In [197]:
def sort_reindex(df):
    df = (df.sort_values(df.columns[0])
          .reset_index(drop=True) ## resets index to be continouus            
         )
    return df

In [27]:
def merge_columns(df, col1, col2, result=None):
    ## noch ein bisschen sauberer wäre gut, zeilenumbruch nach beschreibung vllt und personennamen in richtiger schreibweise
    ## bei result = texts müssten Autoren noch mit den Labels der Personen, auf die verwiesen wird, versehen werden
    
    if result == "zeitraum":
        col3 = "Zeitraum"
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "timespan_" + df["ID"] + "_" + df[col1].apply(str).str[:4] + "_" + df[col2].apply(str).str[:4]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "timespan_" + df["ID"] + "_" + df[col1].apply(str).str[:4]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "timespan_" + df["ID"] + "_" + df[col2].apply(str).str[:4]
        return df
    
    elif result == "season":
        col3 = "Spielzeit"
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "season_" + df["ID"] + "_" + df[col1].apply(str).str[:4] + "_" + df[col2].apply(str).str[:4]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "season_" + df["ID"] + "_" + df[col1].apply(str).str[:4]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "season_" + df["ID"] + "_" + df[col2].apply(str).str[:4]
        return df
    
    elif result == "description":
        col3 = "Beschreibung_Quelle"
        df.loc[df[col1].notna() & df[col2].notna(), col3] = df[col1] + " Quelle: " + df[col2]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = df[col1]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = " Quelle: " + df[col2]
        return df
     
    elif result == "texts":
        col3 = "Texte_Autoren"
        df[col2] = peoplereplace(df[col2])
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "Text(e): " + df[col1] + " Autor(en): " + df[col2]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "Text(e): " + df[col1]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "Autor(en): " + df[col2]
        return df
    
    elif result == "condition":
        col3 = "Zustand_Datum"
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "Zustand: " + df[col1] + " Datum Zustandsaufnahme: " + df[col2].apply(str)
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "Zustand: " + df[col1]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = " Datum Zustandsaufnahme: " + df[col2].apply(str)
        return df
    
    elif result == "institution":
        col3 = "LOC_Institution"
        df.loc[df[col2].notna(), col3] = df[col1].str.replace("LOC_", "G_")
        return df
    else:
        print("Wrong Keyword, nothing happened")
        return df

In [25]:
context = Namespace("http://lod.iti-germany.de/contexts/")
dc = Namespace("http://purl.org/dc/elements/1.1/")
dcterms = Namespace("http://purl.org/dc/terms/")
edm = Namespace("http://www.europeana.eu/schemas/edm/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
nvt = Namespace("http://lod.iti-germany.de/resource/")
nvto = Namespace("http://lod.iti-germany.de/schema/nvto/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
skos = Namespace("http://www.w3.org/2004/02/skos/core#")
wgs84_pos = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")

In [4]:
def bindbind(graph):
    graph.bind("context", "http://lod.iti-germany.de/contexts/")
    graph.bind("dc", "http://purl.org/dc/elements/1.1/")
    graph.bind("dcterms", "http://purl.org/dc/terms/")
    graph.bind("edm", "http://www.europeana.eu/schemas/edm/")
    graph.bind("foaf", "http://xmlns.com/foaf/0.1/")
    graph.bind("nvt", "http://lod.iti-germany.de/resource/")
    graph.bind("nvto", "http://lod.iti-germany.de/schema/nvto/")
    graph.bind("owl", "http://www.w3.org/2002/07/owl#")
    graph.bind("rdf", "http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    graph.bind("rdfs", "http://www.w3.org/2000/01/rdf-schema#")
    graph.bind("skos", "http://www.w3.org/2004/02/skos/core#")
    graph.bind("wgs84_pos", "http://www.w3.org/2003/01/geo/wgs84_pos#")
    return graph

In [5]:
nvt_ds = rdflib.Dataset()
nvt_ds = bindbind(nvt_ds)
pr_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/productions")
ev_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/events")
vid_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/videos")
text_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/texts")
img_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/images")
aud_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/audio")
person_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/persons")
group_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/groups")
loc_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/locations")
city_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/cities")
country_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/countries")
col_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/collections")
series_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/series")
concept_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/concepts")
graph_list = [pr_graph, ev_graph, vid_graph, text_graph, img_graph, aud_graph, person_graph, group_graph, loc_graph, city_graph, country_graph, col_graph, series_graph, concept_graph]

In [191]:
class Sheet():
## name: Name des Reiters
## head: Indikator, wo im df Tabellenüberschriften zu finden sind (0 = Inhalte Zeile 1, 1 = Inhalte Zeile 2, x = Index unverändert)
## df: DataFrame pro Sheet
## tdf: Kopie von df, mit über head definierter Indexanpassung
## tdf_empty_transposed: DataFrame, der nur aus Überschriften (ohne Spaltenachsennamen) besteht, zur Konvertierung nach Excel, inklusive Transponierung
    def __init__(self, name, head, df, graph):
        self.name = name
        self.head = head
        self.df = df
        self.graph = graph
        self.tdf = df.copy()
        self.column_names = None
        self.tdf_empty_transposed = None
        
        if self.head != "x":
            self.tdf.columns = self.tdf.iloc[self.head]
            self.tdf.columns.name = None
        else:
            pass
        if isinstance(self.tdf, pd.DataFrame):
            self.column_names = self.tdf.columns.tolist()
            self.tdf_empty_transposed = self.tdf[0:0].transpose(copy=True)

In [192]:
## Zur Erzeungung von Sheet Instanzen mit Name und Spaltenkopfreferenz als Liste und mit dict Referenz
## column_headers: Liste von 'echten' Spaltenüberschriften, 0, 1, für erste, zweite Zeile, x=Standard
## col_ref dict um nicht mit Zahlen, sondern Namen zu arbeiten
sheet_list = []
col_ref = {}
column_headers = [0, 0, 0, 0, 0, 0, 1, "x", "x", "x", "x", "x", "x"]
for idx, i in enumerate([sheet for sheet in nvt_master.keys()][2:15]):
    sheet_list.append(Sheet(name = i, head = column_headers[idx], df = nvt_master[i], graph = graph_list[idx]))
    col_ref[i] = sheet_list[idx]

In [193]:
## erweitern, um nicht nur zu writen, sondern zukünftig auch upzudaten, und wenn update dann ausgeben: bitte nachtragen, also ein "erwriterer"

## writer = pd.ExcelWriter('Columns_for_Mapping.xlsx', engine='xlsxwriter')
## for i in sheet_list:
##     i.tdf_empty_transposed.to_excel(writer, header=False, sheet_name=i.name)
## writer.save()

In [202]:
col_ref["Produktionen"].tdf = (col_ref["Produktionen"].tdf[col_ref["Produktionen"].tdf['Identifier / geeinigter Name'].str.contains('PR', na=False)] ## drops all rows that don't start with 'PR' or are NaN
                              .drop(col_ref["Produktionen"].tdf[col_ref["Produktionen"].tdf['Identifier / geeinigter Name'].str.contains('PR_Internationaler_Workshop_zur_Biomechanik_GITIS Moskau_Januar_1993', na=False)].index)
                              )
col_ref["Produktionen"].tdf = sort_reindex(col_ref["Produktionen"].tdf) ## das könnte auch für alle auf einmal passieren for tdf sort_reindex


In [216]:
class Mapping():
    
    def __init__(self, df, sheet):
        self.df = df
        self.sheet = sheet

In [219]:
columns_mapped = pd.read_excel('./Mapping_Meta_Py.xlsx', sheet_name=None)

In [232]:
columns_mapped.keys()

odict_keys(['Produktionen', 'Ereignisse', 'Objekte VIDEOS', 'Objekte TEXT', 'Objekte BILD', 'Objekte AUDIO', '||_Personen', '||_Gruppen_Ensembles', '||_Veranstaltungsort', '||_Städte', '||_Länder', 'Sammlungen', 'Serie'])

In [1]:
# print({key:value for key,value in columns_mapped.items()})

In [215]:
mapping_list[0].df

,Spaltenname in Tabelle,In Original,Subject,URI Type,NVTO Property,NVTO Property Label,Object Type,Kommentar
0,Identifier / geeinigter Name,NaN,NaN,nvto:PerformingArtsProduction,NaN,NaN,Reference,NaN
1,Produktionsname / Titel,NaN,NaN,NaN,nvto:hasTitle;skos:prefLabel;rdfs:label,Title + preferred label,Literal,NaN
2,Beschreibung,description1,NaN,NaN,NaN,NaN,NaN,NaN
3,Quelle (Beschreibung),description2,NaN,NaN,NaN,NaN,NaN,NaN
4,Verwendete Texte,texts1,NaN,NaN,NaN,NaN,NaN,theoretisch als InformationObject möglich
5,Autor(en) der Texte,texts2,NaN,NaN,NaN,NaN,NaN,(dann mit Agent)
6,Produktionstyp,NaN,NaN,skos:Concept,nvto:hasType,Has Type,Reference,NaN
7,Beteiligte Gruppen / Compagnies,NaN,NaN,NaN,nvto:hasContributor,Contributor,Reference,NaN
8,Konzept,NaN,NaN,NaN,nvto:hasConceptOriginator,has concept originator,Reference,NaN
9,Textbearbeitung,NaN,NaN,NaN,nvto:hasAdaptor,has adaptor,Reference,NaN
